# 1. Analysis Preparation and Data Cleansing

## 1.1 Set-up environment

In [ ]:
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from matplotlib.ticker import PercentFormatter
plt.rcParams.update({ "figure.figsize" : (8, 5),"axes.facecolor" : "white", "axes.edgecolor":  "black"})
plt.rcParams["figure.facecolor"]= "w"
pd.plotting.register_matplotlib_converters()
pd.set_option('display.float_format', lambda x: '%.2f' % x)

Upload csv file

In [ ]:
df_house = pd.read_csv('data/eda.csv')
# second .csv file uploaded in section 3

## 1.2 Rename and change columns - clean data if necessary

In [ ]:
# rename columns
df_house.rename({'id':'house_id'},axis=1, inplace=True)

In [ ]:
# change date type
df_house['date'] = pd.to_datetime(df_house['date'], format='%Y-%m-%d')
type(df_house['date'][0])

In [ ]:
# limit decimal points
df_house['price'] = df_house['price'].round(2)

In [ ]:
# ZIPCODES
# turn zipcodes into string
df_house['zipcode'] = df_house['zipcode'].astype('string')

In [ ]:
#View
#- No changes
#- NaN means  that we don't know whether change took place

In [ ]:
#WATERFRONT
#- No changes
#- NaN means "we don't know whether property has waterfront view"

In [ ]:
# YR_RENOVATED

# we need to change the year value (i.e., divide by 10)
df_house['yr_renovated'] = df_house.yr_renovated.apply(lambda x: x*0.1)

# we assume that NaN in DB means not yet renovated
df_house['yr_renovated'] = df_house['yr_renovated'].replace(0, np.nan)

In [ ]:
# SQFT_BASEMENT
# assumption is that '0' means 'no basement'
# NaN means that we don't know whether a basement exists

# 2. Data exploration

## 2.1 Property price - descriptive statistics

- mean     540,296.57
- std      367,368.14
- min       78,000.00
- 25%      322,000.00
- 50%      450,000.00
- 75%      645,000.00
- max     7,700,000.00

- There seem to be a lot of outliers. Roughly 200 houses (198) cost more than U>Sd 2 m.
- Median is USD 450000.
- Mode is USD 350000 and USD 450000.
- IQR is USD 323,000
- UP is USD 1,129,500

In [ ]:
print(f'There are {df_house.query("price > 2000000").price.count()} houses more expensive than USD 2m.')
print(f'Median is {df_house.price.median()}.')
print(f'Mode is {df_house.price.mode()}.')


### Property price box diagram

In [ ]:
df_house.query('price < 2000000').price.plot(kind = 'box')

### Property price frequency distribution

In [ ]:
df_house.query('price < 2000000').price.plot(kind = 'hist', bins = 100)

## 2.2 Zipcode-analytics

In [ ]:
df_house['zipcode'].nunique()
# There are 70 different zipcodes in King County

### Zip code and property price

In [ ]:
# Zip codes with the lowest average house prices
df_house.groupby('zipcode').price.describe().sort_values('mean').head(10)

### Ave property prices per zipcode
- averages are based on data 2014 May to 2015 May

In [ ]:
# zip code plot

zip_price_plot = df_house[['zipcode', 'price']].groupby('zipcode').mean('price').sort_values('price').reset_index()



# Set the width of the bars
bar_width = 0.3

# Set positions of the bars
index = np.arange(len(zip_price_plot['zipcode']))

# Plotting the bars
plt.figure(figsize=(12, 6))  # Adjust the figure size as needed
bars1 = plt.bar(index, zip_price_plot['price'], bar_width, label='average price')

# Adding labels and title
plt.xlabel('Zipcode')
plt.ylabel('Average price')
plt.title('Average House price per zipcode')
plt.xticks(index, zip_price_plot['zipcode'], rotation=90)
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()

zip_price_plot

### Insights
- Average property prices are zipcode dependent
- Highest property prices in 98039 (upper-middle class neighborhood)

### Property sales by zipcode area
- averages are based on data 2014 May to 2015 May

In [ ]:
# Zip codes with the highest sales turnover

zip_turnover_plot = df_house[['zipcode','sale_id']].groupby('zipcode').count().sort_values('sale_id').reset_index()

# Set the width of the bars
bar_width = 0.3

# Set positions of the bars
index = np.arange(len(zip_turnover_plot['zipcode']))

# Plotting the bars
plt.figure(figsize=(12, 6))  # Adjust the figure size as needed
bars1 = plt.bar(index, zip_turnover_plot['sale_id'], bar_width, label='turnover')

# Adding labels and title
plt.xlabel('Zipcode')
plt.ylabel('Sales in Period')
plt.title('Sales in period per zipcode')
plt.xticks(index, zip_turnover_plot['zipcode'], rotation=90)
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()

### Insights
- highest turnovers in zipcodes 98103, 98038, 98115 (all upper-middle and middle class white neighborhoods)
- lowest turnovers in zipcode 98039 (upper middle class, small, very high house prices) and 98092 (lower middle class, white)

### Property sales and average property price by zipcode
- averages are based on data 2014 May to 2015 May

In [ ]:
zip_grade_plot = df_house[['zipcode', 'grade']].groupby('zipcode').mean('grade').sort_values('grade').reset_index()
zip_grade_plot.rename({'grade':'grade_average'},axis=1, inplace=True)

merged_zip_plot = pd.merge(zip_price_plot, zip_turnover_plot, on='zipcode', how='inner')
merged_zip_plot.rename({'sale_id':'turnover_in_period'},axis=1, inplace=True)
merged_zip_grade_plot = pd.merge(zip_price_plot, zip_grade_plot, on='zipcode', how='inner')

In [ ]:
# Comparison Average Price and Turnover by Zipcode

# Set the width of the bars
bar_width = 0.3

# Set positions of the bars
index = np.arange(len(merged_zip_plot['zipcode']))

# Plotting the bars
fig, ax1 = plt.subplots(figsize=(12,6))

bars1 = ax1.bar(index - 0.5 * bar_width, merged_zip_plot['price'], bar_width, label='price')

ax1.set_xlabel('zipcode')
ax1.set_ylabel('average house price', color = 'b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.set_xticks(index)
ax1.set_xticklabels(merged_zip_plot['zipcode'], rotation=90)

ax2 = ax1.twinx()
bars2 = ax2.bar(index + 0.5 * bar_width, merged_zip_plot['turnover_in_period'], bar_width, label='turnover_in_period', color = 'red')

ax2.set_ylabel('Turnover in Period', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Adding labels and title
plt.title('Zipcodes by turnover and average sales price')
plt.legend([bars1, bars2], ['price','turnover_in_period'])

# Show the plot
plt.tight_layout()
plt.show()

## 2.3 Property analysis

In [ ]:
property_overview_grade_1 = df_house[['grade','price','bedrooms', 'bathrooms','sqft_living','floors', 'condition', 'yr_built', 'yr_renovated']].groupby('grade').mean()
property_overview_grade_2 = df_house[['grade', 'sale_id']].groupby('grade').count().reset_index()
property_overview_grade_2.rename({'sale_id':'turnover_in_period'},axis=1, inplace=True)
property_overview_grade_1['sqft_price_ave'] = property_overview_grade_1['price'] / property_overview_grade_1['sqft_living']
property_overview_grade = pd.merge(property_overview_grade_1, property_overview_grade_2, on='grade', how='inner')
property_overview_grade
# grade 3 houses tend to have one bedroom. Some of them haven't got separate bedrooms
# grade 5+ tend to have two or more bedrooms
# grade 7+ tend to have more than one floor
# average condition of grade 3 homes tends to be very good; however, they tend not to be renovated (or we don't know)
# houses up to grade 6 tend to be pre-war


### Insights about King County properties
- Highest turnover with grade 4 and 5 properties. Properties have up to 4 bedrooms and 2 bathrooms on average. 
Price per sqft is approx. USD 240.
- Smaller appartments seem to be more expensive in terms of sqft prices.
- Grade 7 and above tend to have more than one bathroom.
- Properties of higher quality tend to be bigger and more expensive in terms of sqft price per living area. They might have a bigger plot size.
- Up to grade 5, properties tend to be either appartments or bungalows. Grade 6 and above tend to be multi-storied.
- Grade 5 to grade 8 properties tend to be of better condition than the rest.
- Grade 3 to grade 6 properties tend to be pre-war, grade 9 and above approx. 25 yrs old.

# 3. Geographical analysis 

## 3.1 Property prices per zipcode

In [ ]:
# upload packages
from urllib.request import urlopen
import json


In [ ]:
# run code
with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/wa_washington_zip_codes_geo.min.json') as response:
    zipcodes = json.load(response)

by_zipcode = df_house.groupby('zipcode').agg({'price': 'mean'}).reset_index()
#by_zipcode['zipcode'] = df_house.zipcode.astype('str')
#by_zipcode.head()

import plotly.express as px


fig = px.choropleth_mapbox(by_zipcode, geojson=zipcodes, locations='zipcode', color='price',
                           color_continuous_scale="Viridis",
                           range_color=(by_zipcode['price'].min(), 1000000),
                           mapbox_style="carto-positron",
                           zoom=8, center = {"lat": 47.553306, "lon": -122.237702},
                           featureidkey="properties.ZCTA5CE10",
                           opacity=0.5,
                           labels={'price':'average house price'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## 3.2 Property size per zipcode
- Number of bedrooms as proxy for property size

In [ ]:
# Average # of bedrooms
with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/wa_washington_zip_codes_geo.min.json') as response:
    zipcodes = json.load(response)

by_zipcode = df_house.groupby('zipcode').agg({'price': 'mean', 'bedrooms': 'mean'}).reset_index()
#by_zipcode['zipcode'] = df_house.zipcode.astype('str')
#by_zipcode.head()

import plotly.express as px


fig = px.choropleth_mapbox(by_zipcode, geojson=zipcodes, locations='zipcode', color='bedrooms',
                           color_continuous_scale="Viridis",
                           range_color=(by_zipcode['bedrooms'].min(), by_zipcode['bedrooms'].max()),
                           mapbox_style="carto-positron",
                           zoom=8, center = {"lat": 47.553306, "lon": -122.237702},
                           featureidkey="properties.ZCTA5CE10",
                           opacity=0.5,
                           labels={'price':'average house price'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Insights
- house sizes in terms of bedrooms smaller in adjacent inner city zipcode areas
- bigger houses in 98039 and 98040, and 98004, 98005, and 98006

## 3.3 Property grade per zipcode

In [ ]:
new_zip_price_grade = merged_zip_grade_plot

# run code
with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/wa_washington_zip_codes_geo.min.json') as response:
   zipcodes = json.load(response)

by_zipcode = merged_zip_grade_plot
#by_zipcode['zipcode'] = df_house.zipcode.astype('str')
#by_zipcode.info()

import plotly.express as px


fig = px.choropleth_mapbox(by_zipcode, geojson=zipcodes, locations='zipcode', color='grade_average',
                           color_continuous_scale="Viridis",
                           range_color=(by_zipcode['grade_average'].min(), by_zipcode['grade_average'].max()),
                           mapbox_style="carto-positron",
                           zoom=8, center = {"lat": 47.553306, "lon": -122.237702},
                           featureidkey="properties.ZCTA5CE10",
                           opacity=0.5,
                           labels={'grade_average':'average house grade'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Insights
- average house grades seem to be lower south of Seattle city center
- average house grades seem to be higher in water-faceing zipcodes than in others
- average house grades are higher towards the East, especially the North East 
- Zipcode 98039, primarily in Medina, the highest average house grade (upper middle class area, predominantly white, household income $208K)
- Zipcode 98168 (Tukwila, Burien, SeaTac, Boulevard Park, and White Center) has the lowest average house grade (lower middle class, mixed, household income of $71K)

## 3.4 Changes in price per sqft_living by grade and zipcode
- Assumption is that the change in price per "sqft living area" is a good proxy for general price changes across grade-based property categories

### Changes in price per sqft_living by grade

In [ ]:

comp_1 = df_house[['grade','date', 'price', 'sqft_living']].query('date >= "2014-05-01" and date <= "2014-7-31"').groupby('grade').mean('price')
comp_1.rename({'price':'price2014', 'sqft_living': 'sqft_living2014'},axis=1, inplace=True)


comp_2 = df_house[['grade','date', 'price', 'sqft_living']].query('date >= "2015-03-01" and date <= "2015-5-31"').groupby('grade').mean('price')
comp_2.rename({'price':'price2015', 'sqft_living': 'sqft_living2015'},axis=1, inplace=True)

comp_1['price_sqft_ave_2014MJ'] = comp_1['price2014'] / comp_1['sqft_living2014']
comp_2['price_sqft_ave_2015MM'] = comp_2['price2015'] / comp_2['sqft_living2015']

merged_comp_range = pd.merge(comp_1, comp_2, on='grade', how='inner')

merged_comp_range['delta_price_in_%'] = ((merged_comp_range['price_sqft_ave_2015MM']/merged_comp_range['price_sqft_ave_2014MJ'])-1)*100
merged_comp_range

### Insights
- Assumption is that average price per sqft living area is a good indication for price changes.
- Property prices rose significantly between the period May to July 2014 and March to May 2015 but not for all grades.
- Super mansion prices stayed the same, while prices for high end luxury homes rose by 34%.
- Grade 4, grade 5 and grade 6 average property prices rose significantly (14% and 23%).

### Changes in price per sqft_living by zipcode
- Assumption is that the change in price per "sqft living area" is a good proxy for general price changes across zipcodes (better reflection of property price deltas than vartiable "price")

In [ ]:
comp_zip_1 = df_house[['zipcode','date', 'price', 'sqft_living']].query('date >= "2014-05-01" and date <= "2014-7-31"').groupby('zipcode').mean('price').reset_index()
comp_zip_1.rename({'price':'price2014', 'sqft_living': 'sqft_living2014'},axis=1, inplace=True)

comp_zip_2 = df_house[['zipcode','date', 'price', 'sqft_living']].query('date >= "2015-03-01" and date <= "2015-5-31"').groupby('zipcode').mean('price').reset_index()
comp_zip_2.rename({'price':'price2015', 'sqft_living': 'sqft_living2015'},axis=1, inplace=True)

comp_zip_1['price_sqft_ave_2014MJ'] = comp_zip_1['price2014'] / comp_zip_1['sqft_living2014']
comp_zip_2['price_sqft_ave_2015MM'] = comp_zip_2['price2015'] / comp_zip_2['sqft_living2015']

merged_comp_zip = pd.merge(comp_zip_1, comp_zip_2, on='zipcode', how='inner')

merged_comp_zip['delta_price_in_%'] = ((merged_comp_zip['price_sqft_ave_2015MM']/merged_comp_zip['price_sqft_ave_2014MJ'])-1)*100

In [ ]:
df_zip = merged_comp_zip

fig = px.choropleth_mapbox(df_zip, geojson=zipcodes, locations='zipcode', color='delta_price_in_%',
                            color_continuous_scale="Viridis",
                            range_color=(df_zip['delta_price_in_%'].min(),df_zip['delta_price_in_%'].max()),
                            mapbox_style="carto-positron",
                            zoom=8, center = {"lat": 47.553306, "lon": -122.237702},
                            featureidkey="properties.ZCTA5CE10",
                            opacity= 0.5,
                            labels={'delta_price_in_%':'delta in ave house prices 2014 Q2 to 2015 Q2'}
                            )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

### Insights
- Prices in zipcode 98125 rose significantly during period (27%).
- Prices in zipcodes in the South rose less than prices in zipcodes in the North East.
- Prices in inner borough zipcodes towards the N, S and SE of Seattle centre remained flat (98108, 98109, 98144).
- Prices rose more in "outer" inner borough zipcode to the N and S of Seattle Center (e.g. 98118, 98168, 98188)

## 3.5 Median income per neighborhood
- dataframe df_house3 is based on seattle_income_zip.csv, which contains median household income data for Seattle plus King County
- input for Nicole Johnson analysis

In [ ]:
df_house3 = pd.read_csv('data/seattle_income_zip.csv')

df_house3.rename({'ZIP Code':'zipcode', 'Median Household Income(2021)': 'median_income'},axis=1, inplace=True)
df_house3['zipcode'] = df_house3['zipcode'].astype('string')

In [ ]:
df_zip = df_house3

fig = px.choropleth_mapbox(df_zip, geojson=zipcodes, locations='zipcode', color='median_income',
                            color_continuous_scale="Viridis",
                            range_color=(df_zip['median_income'].min(),df_zip['median_income'].max()),
                            mapbox_style="carto-positron",
                            zoom=8, center = {"lat": 47.553306, "lon": -122.237702},
                            featureidkey="properties.ZCTA5CE10",
                            opacity= 0.5,
                            labels={'median_income':'median_income_by_zip'}
                            )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# 4. Analysis for Nicole Johnson
## 4.1 Main table for plotting median income by zipcode
- Reference dataframe for all median income zipcodes that might form part of analyis
- Contains zipcodes of "preferred areas" and "adjacent zipcodes"
- df_house3_1 is based on df_house3

In [ ]:
df_house3_1 = df_house3.query('zipcode in ["98101", "98102", "98103", "98104", "98105", "98107", "98121", "98122", "98109", "98112", "98115", "98119", "98117", "98121", "98125", "98133", "98177", "98199"]')

## 4.2 Main table for identifying properties by zipcode
- df_trendy is a zipcode-based subset of df_house2. It contains all property information that is relevant for zipcodes relevant for the Nicole Johnson search (preferred and adjacent) 

In [ ]:
df_trendy = df_house.query('zipcode in ["98101", "98102", "98103", "98104", "98105", "98107", "98121", "98122", "98109", "98112", "98115", "98119", "98117", "98121", "98177", "98199"]')

## 4.3 Identify prices and property sales numbers in Nicole's preferred areas
- merged table allows to identify target zipcode

In [ ]:
df_trendy1 = df_trendy[['zipcode', 'price', 'bedrooms']].groupby('zipcode').mean()
df_trendy2 = df_trendy[['zipcode', 'sale_id']].groupby('zipcode').count().reset_index()
df_trendy2.rename ({'sale_id':'turnover'},axis=1, inplace=True)
df_trendy_merge = pd.merge(df_trendy1, df_trendy2, on='zipcode', how='inner').sort_values("price")

df_trendy_merge

## 4.4 Graphic analysis of Nicole's preferred zipcodes (average price and turnover)

In [ ]:
df_trendy_merge.query('zipcode in ["98101", "98102", "98103", "98104", "98105", "98107", "98121", "98122"]')

In [ ]:
df_trendy_merge_preferred = df_trendy_merge.query('zipcode in ["98101", "98102", "98103", "98104", "98105", "98107", "98121", "98122"]')

df_zip = df_trendy_merge_preferred

fig = px.choropleth_mapbox(df_zip, geojson=zipcodes, locations='zipcode', color='price',
                            color_continuous_scale="Viridis",
                            range_color=(df_zip['price'].min(),df_zip['price'].max()),
                            mapbox_style="carto-positron",
                            zoom=8, center = {"lat": 47.553306, "lon": -122.237702},
                            featureidkey="properties.ZCTA5CE10",
                            opacity= 0.5,
                            labels={'price':'average_price_by_zip'}
                            )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

## 4.5 Identification of additional search areas 
- zipcode areas are visualized by median income per zipcode
- we select zipcodes with median household income > USD 100K

In [ ]:
df_trendy_merge_selected = df_house3_1.query('zipcode in ["98109", "98112", "98115", "98119", "98117", "98121", "98125", "98133", "98144", "98177", "98199"]')


df_zip = df_trendy_merge_selected

fig = px.choropleth_mapbox(df_zip, geojson=zipcodes, locations='zipcode', color='median_income',
                            color_continuous_scale="Viridis",
                            range_color=(df_zip['median_income'].min(),df_zip['median_income'].max()),
                            mapbox_style="carto-positron",
                            zoom=8, center = {"lat": 47.553306, "lon": -122.237702},
                            featureidkey="properties.ZCTA5CE10",
                            opacity= 0.5,
                            labels={'median_income':'median_income_by_zip'}
                            )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

## 4.6 Identification of properties in selected zipcodes
We need a two bedroom in zipcode 98117, 98103, 98107, 98155

In [ ]:
df_house.query('(zipcode in ["98117", "98103", "98107", "98155"]) and (price < 450000) and (bedrooms == 2) and (bathrooms == 2)')